# Run an OpenMM simulation with NanoVer

NanoVer can run OpenMM simulations in two ways: either using [ASE as an interface](../ase/openmm_nanotube.ipynb), or directly using OpenMM mechanisms. Using the ASE interface offers the most flexibility to customise a workflow; in the [graphene example](..ase/openmm_graphene.ipynb) we control the physics parameter of a running simulation from a jupyter notebook. The ASE interface misses some specific OpenMM feature, though; the most noticeable one being holonomic constraints. Using the OpenMM mechanisms without ASE as an interface gives access to all of OpenMM features, but may require more work for some customisation needs. Here, we demonstrate how to to use the OpenMM mechanisms.

## Prepare an OpenMM simulation

Running an OpenMM simulation with NanoVer requires 2 elements: an OpenMM simulation and a NanoVer runner. The simulation is a normal OpenMM simulation that you can prepare how you would normally do. The only constraint is to include the NanoVer custom force. Especially with small system, removing the center of mass motion leads to unintuitive behaviours when you interact with the molecules.

Here, we prepare a simulation of a polyalanine following instructions adaption from [the OpenMM documentation](http://docs.openmm.org/7.0.0/userguide/application.html#a-first-example).

In [ ]:
from simtk import openmm as mm
from simtk.openmm import app
from simtk import unit

from nanover.openmm.imd import add_imd_force_to_system

pdb = app.PDBFile('openmm_files/17-ala.pdb')
forcefield = app.ForceField('amber99sb.xml', 'tip3p.xml')
system = forcefield.createSystem(
    pdb.topology,
    nonbondedMethod=app.PME,
    nonbondedCutoff=1 * unit.nanometer,
    constraints=app.HBonds,
    removeCMMotion=False,
)

# Add the NanoVer custom force to the system. This force
# is used by NanoVer to transmit the force from the VR
# controlers.
# The force *must* be added *before* the system is attached
# to a simulation.
add_imd_force_to_system(system)

integrator = mm.LangevinIntegrator(
    300 * unit.kelvin,
    1 / unit.picosecond,
    0.002 * unit.picoseconds,
)
simulation = app.Simulation(pdb.topology, system, integrator)
simulation.context.setPositions(pdb.positions)

The simulation is a normal OpenMM simulation and can be used as such. The NanoVer custom force should not impact the use of the simulation outside of NanoVer.

In [ ]:
simulation.minimizeEnergy()
simulation.step(100)

## Use the simulation with NanoVer

NanoVer works with a client-server architecture. A NanoVer runner creates the server and make the link between that server and the simulation. Here, we create a runner and attach to it the simulation we created earlier. Note that the runner adds a reporter to the simulation.

In [ ]:
from nanover.openmm import OpenMMRunner
runner = OpenMMRunner(simulation)
runner.run()

From that point you have the simulation running and a server waiting for clients to connect.

Once you are done, you can close the server to free the network port.

In [ ]:
runner.close()

## Save a simulation to file

Once you have a simulation ready, you may want to save this setup on a file. By doing so, it becomes simpler to reuse the simulation, including with the `nanover-omm-server` command line tool.

`nanover.openmm.serializer.serialize_simulation` creates an XML that describes the system, the initial structure, and the integrator. `nanover.openmm.serializer.deserialize_simulation` reads such XML to produce a simulation object.

In [ ]:
from nanover.openmm import serializer
with open('simulation.xml', 'w') as outfile:
    outfile.write(serializer.serialize_simulation(simulation))

In [ ]:
with open('simulation.xml') as infile:
    simulation_2 = serializer.deserialize_simulation(infile.read())

## Use a saved simulation

With a simulation saved as an XML file, setting up a NanoVer runner becomes much simpler:

In [ ]:
from nanover.openmm import OpenMMRunner
runner = OpenMMRunner.from_xml_input('openmm_files/17-ala.xml')
runner.run()

In [ ]:
# Close the server when done with it.
runner.close()

## Save the trajectory

The first benefit of iMD-VR is to see and interect with molecular systems, it is sometimes usefull to save the trajectory as well to run analyses or latter stages of a workflow. Saving the trajectory is done in the regular way for OpenMM simulations: by attaching a reporter. Here we attach a DCD reporter to save the trajectory in the DCD format every 500 frames.

In [ ]:
from nanover.openmm import OpenMMRunner
runner = OpenMMRunner.from_xml_input('openmm_files/17-ala.xml')

dcd_reporter = app.DCDReporter('output.dcd', 500)
simulation = runner.simulation
simulation.reporters.append(dcd_reporter)

runner.run()

In [ ]:
# Close the server when done with it.
runner.close()

The DCD reporter does not close the file when the simulation is finished. In some cases, this can prevent to open the trajectory with an other software as long as the jupyter kernel is running. This line closes the file. Note that this will break the reporter in case you want to continue running the simulation.

In [ ]:
dcd_reporter._dcd._file.close()